In [2]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [1]:
import pandas as pd
from scraper_functions import has_no_words
import re

In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# stopwords.remove('no')
# stopwords.remove('not')
fil_stopwords = ['ang', 'po', 'naman', 'ito', 'medyo', 'may', 'yung', 'kasi', 'sa', 'na', 'ng', 'ko', 'ba', 'ako', 'lang']
for word in fil_stopwords:
    stopwords.append(word)

In [4]:
file_name = 'tika_transcript'

df = pd.read_csv('data/{}.csv'.format(file_name), index_col=0)
df.shape

(455, 3)

In [5]:
df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')
df = df.sort_values('date', ascending=False).reset_index().drop('index', axis=1)
df.head(10)

,title,date,text
0,Talk to the People of President Rodrigo Roa Du...,2020-10-05,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Talk to the People of President Rodrigo Roa Du...,2020-09-28,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,Statement of President Rodrigo Roa Duterte dur...,2020-09-22,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Talk to the People of President Rodrigo Roa Du...,2020-09-21,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,Talk to the People of President Rodrigo Roa Du...,2020-09-14,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,Talk to the People of President Rodrigo Roa Du...,2020-09-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,Statement of President Rodrigo Roa Duterte at ...,2020-09-02,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,Speech of President Rodrigo Roa Duterte during...,2020-08-31,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
8,Talk to the People of President Rodrigo Roa Du...,2020-08-31,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
9,Talk to the People of President Rodrigo Roa ...,2020-08-25,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [6]:
# Example of speech; remove everything before the date
entry = df.loc[150]
sample = entry['text']
index = sample.find(']')
print(entry['title'])
print(sample[index+1:1000])
print('===============================')
print(sample[-1000:])

Speech of President Duterte during the mass oath taking of the newly elected officers of the Federation of Filipino-Chinese Chambers of Commerce and Industry, Inc. (FFCCCII)
 
  
  
Ladies and gentlemen, I am very proud to administer your oath of office. 
You choose the venue and good evening ma’am, President Ramos and... 
  
[oath-taking ceremony] 
  
Maupo muna kayo sandali. I was told just to administer your oath and that’s 
it. Eh sabi ko, just --- tatlo ito, the mayors and governors and ito without 
speeches. 
  
Sabi ko, look I did study foreign service, totoo ‘yan. Hindi dini-discredit. I 
took up diplomatic practice and procedure. So it go
d sabi ko sa 
iyo, I will protect you. I will protect not only Chinese. Everybody. Whether 
it's German or doing business dito. 
  
Ayaw ko talaga ng kalokohan. Ayaw ko 'yang mag sige... And 'yung 
mayroon kayo diyan na historical na 'yang hingi niya, tell me para 'yung 
mga anak ninyo balang araw aabutin pa 'yan eh, hanggang 30 years 'yan 
s

## Cleaning the Data

In [7]:
def clean_data(text):
    titles = ['mr', 'dr', 'mrs', 'atty', 'ms', 'inc']
    titles_str = ''.join([f'(?<!({x}))' for x in titles])
    
    if not isinstance(text, float):
        # Remove headings like "PRESIDENT DUTERTE:"
        text = re.sub(r'[A-Z ]+: ?', '', text)
        
        text = text.lower()
        
        # Remove words inside brackets, e.g. [laughter], [applause]
        text = re.sub(r'\[.*?\]', '', text)
        
#         Remove punctuation
#         I included the asterisk * to include swears
        text = re.sub(r'([^A-Z a-z*\'\-.!?])', '', text)
    
#         Remove hyphens EXCEPT if they're preceded by a letter
#         or if they are group as three hyphens
        text = re.sub(r'(?<!-)(-{1,2}|-{4,})(?!-)', '', text)

#         Not sure if I will still remove stopwords
#         text = ' '.join([word for word in text.split() if word not in stopwords])

        text = text.replace('presidential news desk', '')
        text = text.replace('pcoopnd', '')
        
#         Remove puncutation
        text = re.sub(r'\.{3}', ' ', text)
        text = text[:-1]
#         period_reg = titles_str + '\. '
#         text = re.sub(period_reg, ' . ', text)
#         text = re.sub('\? ', ' ? ', text)
#         text = re.sub('! ', ' ! ', text)
    
    return text

In [244]:
def replace_swears(text):
    if not isinstance(text, float):
        text = text.replace('kat*********', 'katarantaduhang')
        text = text.replace('k*********', 'katarantaduhan')
        text = text.replace('ka****han', 'kagaguhan')
        text = text.replace('ka*******', 'kagaguhan')
        text = text.replace('k********', 'kagaguhan')
        text = text.replace('t**********', 'tarantadong')
        text = text.replace('t********', 'tarantado')
        text = text.replace('t*****i***', 'tanginang')
        text = text.replace('t****i***', 'tanginang')
        text = text.replace('t*** i*****', 'tanginang')
        text = text.replace('t*** i****', 'tangina')
        text = text.replace('t*** i***', 'tangina')
        text = text.replace('t****i**', 'tangina')
        text = text.replace('t****i**', 'tangina')
        text = text.replace('t****** i****', 'tanginang')
        text = text.replace('t****i***', 'tanginang')
        text = text.replace('t***** i**', 'tangina')
        text = text.replace('t**** i***', 'tangina')
        text = text.replace('t**** i**', 'tangina')
        text = text.replace('t*** i**', 'tangina')
        text = text.replace('t*******', 'tarantado')
        text = text.replace('t******', 'tarantado')
        text = text.replace('t*****', 'taragis')
        text = text.replace('t****', 'tubol')
        text = text.replace('t***', 'tangina')
        text = text.replace('t**', 'tae')
        text = text.replace('magp*********', 'mag-puputa-puta')
        text = text.replace('p********* i**', 'putangina')
        text = text.replace('p******** i**', 'putangina')
        text = text.replace('p********', 'putangina')
        text = re.sub(r't{1}\*+ ?i\*{3,}', 'tanginang', text)
        text = re.sub(r'p{1}\*+ ?i\*{3,}', 'putanginang', text)
        text = re.sub(r't{1}\*+ ?i\*{1,2}', 'tangina', text)
        text = re.sub(r'p{1}\*+ ?i\*{1,2}', 'putangina', text)
        text = text.replace('k*** i**', 'kingina')
        text = text.replace('l*****i****', 'lecheleche')
        text = re.sub(r'l\*{6,}', 'lecheng', text)
        text = re.sub(r'l\*{2,5}', 'leche', text)
        text = text.replace('mam***** i**', 'mamutangina')
        text = text.replace('mamutangi**', 'mamutangina')
        text = text.replace('q** s* j***', 'que se joda')
        text = text.replace('y******', 'yaring')
        text = text.replace('y*****', 'yaring')
        text = text.replace('y****', 'yari')
        text = text.replace('y***', 'yari')
        text = text.replace('kag*******', 'kagaguhang')
        text = text.replace('kap*******', 'kaputahan')
        text = text.replace('kap******', 'kaputahan')
        text = text.replace('pinu**** i**', 'pinutangina')
        text = text.replace('p*******', 'putangina')
        text = text.replace('p******', 'puchang')
        text = text.replace('p*****', 'putang')
        text = text.replace('p****', 'puta')
        text = text.replace('p***', 'puta')
        text = text.replace('p**', 'puta')
        text = text.replace('p*', 'puta')
        text = text.replace('i*****', 'idiots')
        text = text.replace('i****', 'idiot')
        text = text.replace('i***', 'igit')
        text = text.replace('god d******', 'goddamn')
        text = text.replace('g**d****', 'goddamn')
        text = text.replace('g** d***', 'goddamn')
        text = text.replace('g**d***', 'goddamn')
        text = text.replace('godd*****', 'goddamn')
        text = text.replace('godd****', 'goddamn')
        text = text.replace('god****', 'goddamn')
        text = text.replace('g**d****', 'goddamn')
        text = text.replace('g********', 'goddamn')
        text = text.replace('g*******', 'goddamn')
        text = text.replace('g******', 'goddamn')
        text = text.replace('g*****', 'gagong')
        text = text.replace('g****', 'gago')
        text = text.replace('o***', 'tite')
        text = text.replace('g***', 'gago')
        text = text.replace('g**o', 'gago')
        text = text.replace('g**a', 'gaga')
        text = text.replace('b**** s**** i**', 'bullshitting')
        text = text.replace('b**********', 'bullshitting')
        text = text.replace('b*** s*****', 'bullshitted')
        text = text.replace('b*b*** s******', 'bullshit')
        text = text.replace('b**s***ted', 'bullshitted')
        text = text.replace('b*** s***', 'bullshit')
        text = text.replace('b*** s**', 'bullshit')
        text = text.replace('b*********', 'binullshit')
        text = text.replace('bin****s***', 'binullshit')
        text = text.replace('b*******', 'bullshit')
        text = text.replace('b******', 'bitches')
        text = text.replace('b*****', 'bobong')
        text = text.replace('s** of a b****', 'bitch')
        text = text.replace('b****', 'bitch')
        text = text.replace('b***', 'bobo')
        text = text.replace('s********', 'shitting')
        text = text.replace('s*****', 'shitty')
        text = text.replace('s***', 'shit')
        text = text.replace('a******', 'asshole')
        text = text.replace('a*****', 'asshole')
        text = text.replace('a**h***', 'asshole')
        text = text.replace('a**', 'ass')
        text = text.replace('d***', 'damn')
        text = text.replace('u******', 'ukinam')
        text = text.replace('u*****', 'ukinam')
        text = text.replace('u***', 'ulol')
        text = text.replace('v*****', 'vagina')
        text = text.replace('f*** y**', 'fuck you')
        text = re.sub(r'f{1}\*{5,}', 'fucking', text)
        text = re.sub(r'f{1}\*{2,4}', 'fuck', text)
    
    return text

In [38]:
from nltk import tokenize

# Split the text into a list of sentences
def split_sentences(text):
    if not isinstance(text, float):
        index_of_date = text.find(']')
        text = text[index_of_date+1:]
        text = ' '.join(text.split())
        return tokenize.sent_tokenize(text)
    
    return []

In [245]:
sentences = df['text'].apply(split_sentences).sum()
df_clean = pd.DataFrame(sentences, columns=['text'])
df_clean['clean_text'] = df_clean['text'].apply(clean_data)
df_clean = df_clean.drop_duplicates()
df_clean['clean_text'] = df_clean['clean_text'].apply(replace_swears)

In [12]:
import random
index = random.randint(0, df.shape[0])
print('INDEX: ' + str(index))
' '.join(df_clean.loc[index]['clean_text'].split()[:300])

INDEX: 257


'so because we have prepared well the train law the rice tarrification law and all the laws that you --- you sponsored have helped us in managing this --- this cri --- this crisis'

In [255]:
df_clean.loc[135317]['text']

'Pala-away, palahingi, pero not necessarily „yung word na w****.'

In [246]:
df_clean.iloc[105743:][df_clean['clean_text'].str.contains(r'[A-Za-z]{1}\*+', regex=True)]

<ipython-input-246-f62bc4909864>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean.iloc[105743:][df_clean['clean_text'].str.contains(r'[A-Za-z]{1}\*+', regex=True)]


,text,clean_text
111819,‘Di nagk****-k**** ‘yung mga [laughter] singer...,di nagk****k**** yung mga singer niya pati ano
111823,Tapos… Wala namang --- ang sayaw kasi ngayon d...,tapos wala namang --- ang sayaw kasi ngayon da...
112808,What the f***** s**?,what the fucking s**
117858,Ako unsa ma’y mawa sa akong k****?,ako unsa may mawa sa akong k****
122810,S*******.,shitty**
133087,Ako pa’y pasanginlan anang b**** s*** i** niya.,ako pay pasanginlan anang bitch shit i** niya
133131,B**** s**** i**.,bitch shit* i**
133300,It started with the extrajudicial killing and ...,it started with the extrajudicial killing and ...
134209,God d******.,god damn***
134376,"Nag- Presidente ako, u******.",nag presidente ako ukinam*


In [ ]:
# Remove translations

## Saving cleaned text as CSV

In [70]:
# Remove NaN values
df_clean = df_clean.dropna()

In [71]:
df_clean.to_csv('data/{}_sentences.csv'.format(file_name))
df_clean.head()

,text,clean_text
0,"PRESIDENT RODRIGO ROA DUTERTE: Good day, my co...",good day my countrymen
1,We are here gathered to render a report weekly...,we are here gathered to render a report weekly...
2,Kaya kami nandito at marami namang haka-haka p...,kaya kami nandito at marami namang hakahaka pa...
3,"Gawain mo na, masama ka pa rin.",gawain mo na masama ka pa rin
4,"Kung hindi mo gawin, mas lalo kang masama.",kung hindi mo gawin mas lalo kang masama


In [72]:
df_clean.shape

(174346, 2)